In [36]:
from gekko import GEKKO
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import numpy as np

Importando dados

In [37]:
#importar csv - INDICADORES
dados_trimestrais = pd.read_csv('data\\base_dados_tri_2024.csv', sep=';')
dados_trimestrais['DT_FIM_EXERC'] = pd.to_datetime(dados_trimestrais['DT_FIM_EXERC'])
dados_trimestrais['VL_CONTA'] = dados_trimestrais['VL_CONTA'].astype(float)
#Colunas no dataframe índice, CD_CVM, CD_CONTA, ESCALA_MOEDA, DT_INI_EXERC, DS_CONTA, VL_CONTA, ST_CONTA_FIXA)

#importar csv - INDÍCES =
ipca = pd.read_csv('data\\ipca.csv', sep=';')
#definir index ipca como coluna Unnamed: 0
ipca = ipca.set_index('DATA')
#transformar index em datetime
ipca.index = pd.to_datetime(ipca.index, format='%Y-%m-%d')
ipca['IPCA'] = ipca['IPCA'].astype(float)
print(ipca.head())

selic = pd.read_csv('data\\selic.csv', sep=';')
selic = selic.set_index('DATA')
selic.index = pd.to_datetime(selic.index, format='%Y-%m-%d')
selic['SELIC'] = selic['SELIC'].astype(float)
print(selic.head())

igpm = pd.read_csv('data\\igpm.csv', sep=';')
igpm = igpm.set_index('DATA')
igpm.index = pd.to_datetime(igpm.index, format='%Y-%m-%d')
igpm['IGPM'] = igpm['IGPM'].astype(float)
print(igpm.head())

vix_data = pd.read_csv('data\\VIX.csv', header=0, sep=",")
vix = vix_data[['DATE', 'CLOSE']]
vix = vix.set_index('DATE')
vix.index = pd.to_datetime(vix.index)
vix['CLOSE'] = vix['CLOSE'].astype(float)
print(vix.head())

#incc
incc = pd.read_csv('data\\INCC.csv', sep=',', index_col=0, header=0)
incc.index = pd.to_datetime(incc.index, format='%Y-%m-%d')
incc = incc.drop(incc.index[0])
incc['Valor'] = incc['Valor'].astype(float)
print(incc.tail())


#IEE
iee = pd.read_csv('data\\iee.csv', sep=';', index_col=0, header=0)
iee.index = pd.to_datetime(iee.index, format='%Y-%m-%d')
iee['Valor'] = iee['Valor'].str.replace('.', '').str.replace(',', '.').astype(float)
print(iee.tail())


# Get data on this ticker
tickerData = yf.Ticker('^BVSP')
# Get the historical prices for this ticker with monthly interval
tickerDf = tickerData.history(period='1mo', start='2010-01-01', end='2024-12-31')
# Deixar apenas data no indíce
tickerDf.index = tickerDf.index
ibov = tickerDf[['Close']]
ibov = ibov.rename_axis('DATE')
ibov.index = ibov.index.tz_localize(None)
ibov['Close'] = ibov['Close'].astype(float)
print(ibov.head())

dict_indices = {'ipca': ipca, 'selic': selic, 'igpm': igpm, 'vix': vix, 'ibov': ibov, 'incc': incc, 'iee': iee}



            IPCA
DATA            
1981-01-01  6.82
1981-02-01  6.15
1981-03-01  5.10
1981-04-01  5.04
1981-05-01  7.39
            SELIC
DATA             
1996-01-01    0.0
1996-02-01    0.0
1996-03-01    0.0
1996-04-01    0.0
1996-05-01    0.0
            IGPM
DATA            
1989-01-01   0.0
1989-02-01   0.0
1989-03-01   0.0
1989-04-01   0.0
1989-05-01   0.0
            CLOSE
DATE             
1990-01-02  17.24
1990-01-03  18.19
1990-01-04  19.22
1990-01-05  20.11
1990-01-08  20.26
            Valor
Data             
2024-02-01   0.13
2024-03-01   0.28
2024-04-01   0.52
2024-05-01   0.86
2024-06-01   0.71
               Valor
Data                
2024-02-01  91201.24
2024-03-01  88654.31
2024-04-01  85294.00
2024-05-01  87311.19
2024-06-01  88289.36
              Close
DATE               
2010-01-04  70045.0
2010-01-05  70240.0
2010-01-06  70729.0
2010-01-07  70451.0
2010-01-08  70263.0


In [38]:
date_ranges = {}

for key, data in dict_indices.items():
    try:
        # A primeira coluna é a data, portanto pegamos a mínima e máxima data
        min_date = data.index.min()
        max_date = data.index.max()
        date_ranges[key] = (min_date, max_date)
    except Exception as e:
        date_ranges[key] = f"Error: {e}"
        print(f"Error: {e}")
        break

# Identificar o maior intervalo de tempo comum entre todos os arquivos (Janeiro de 1981 a Dezembro de 2023)
print(date_ranges)
common_start = max([date_range[0] for date_range in date_ranges.values()])
common_end = min([date_range[1] for date_range in date_ranges.values()])

print(f" Intervalo comum de {common_start} até {common_end}")


{'ipca': (Timestamp('1981-01-01 00:00:00'), Timestamp('2024-06-01 00:00:00')), 'selic': (Timestamp('1996-01-01 00:00:00'), Timestamp('2024-07-01 00:00:00')), 'igpm': (Timestamp('1989-01-01 00:00:00'), Timestamp('2024-06-01 00:00:00')), 'vix': (Timestamp('1990-01-02 00:00:00'), Timestamp('2024-06-28 00:00:00')), 'ibov': (Timestamp('2010-01-04 00:00:00'), Timestamp('2024-08-09 00:00:00')), 'incc': (Timestamp('1944-02-01 00:00:00'), Timestamp('2024-06-01 00:00:00')), 'iee': (Timestamp('1997-01-01 00:00:00'), Timestamp('2024-06-01 00:00:00'))}
 Intervalo comum de 2010-01-04 00:00:00 até 2024-06-01 00:00:00


ESTUDO DE FILTRO PASSA BAIXA PARA EVITAR ALIASING - IBOV

In [39]:
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.signal import butter, lfilter


#usando filtro passa baixa para retirar efeito de aliasing
def butter_lowpass_filter(data):
    cutoff = 1/63 # frequência trimestral
    fs = 1 # frequência diária
    order = 5 # ordem de filtro intermediária 
    nyquist = 0.5 * fs
    normal_cutoff = cutoff / nyquist
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    y = lfilter(b, a, data)
    print("Filtro executado com sucesso")
    return y



In [40]:
#resample usando média para suavizar o ruído -> fazer um alising
resampled_indices = {}

def ajuste_indice(indice):
    print(f"Iteração: {key}")
    new_indice = butter_lowpass_filter(indice)
    new_indice = indice.resample('Q-MAR').last()

    return new_indice

for key, data in dict_indices.items():
    resampled_indices[key] = ajuste_indice(data)
    print(resampled_indices[key].tail())


Iteração: ipca
Filtro executado com sucesso
            IPCA
DATA            
2023-06-30 -0.08
2023-09-30  0.26
2023-12-31  0.56
2024-03-31  0.16
2024-06-30  0.21
Iteração: selic
Filtro executado com sucesso
            SELIC
DATA             
2023-09-30  12.75
2023-12-31  11.75
2024-03-31  10.75
2024-06-30  10.50
2024-09-30  10.50
Iteração: igpm
Filtro executado com sucesso
            IGPM
DATA            
2023-06-30 -1.93
2023-09-30  0.37
2023-12-31  0.74
2024-03-31 -0.47
2024-06-30  0.81
Iteração: vix
Filtro executado com sucesso
            CLOSE
DATE             
2023-06-30  13.59
2023-09-30  17.52
2023-12-31  12.45
2024-03-31  13.01
2024-06-30  12.44
Iteração: ibov
Filtro executado com sucesso
               Close
DATE                
2023-09-30  116565.0
2023-12-31  134185.0
2024-03-31  128106.0
2024-06-30  123907.0
2024-09-30  130615.0
Iteração: incc
Filtro executado com sucesso
            Valor
Data             
2023-06-30   0.71
2023-09-30   0.34
2023-12-31   0.07
2024-03-3

In [41]:
# Get the time window from the 'DT_FIM_EXERC' column in dados_trimestrais
start_date = dados_trimestrais['DT_FIM_EXERC'].min()
#somar 1 mês para pegar o próximo trimestre	
end_date = dados_trimestrais['DT_FIM_EXERC'].max()
print(start_date, end_date)

#lign the indices in resampled_indices with the same time window
for key, indice in resampled_indices.items():
    resampled_indices[key] = indice.loc[start_date:end_date]

resampled_indices

2011-03-31 00:00:00 2024-05-31 00:00:00


{'ipca':             IPCA
 DATA            
 2011-03-31  0.79
 2011-06-30  0.15
 2011-09-30  0.53
 2011-12-31  0.50
 2012-03-31  0.21
 2012-06-30  0.08
 2012-09-30  0.57
 2012-12-31  0.79
 2013-03-31  0.47
 2013-06-30  0.26
 2013-09-30  0.35
 2013-12-31  0.92
 2014-03-31  0.92
 2014-06-30  0.40
 2014-09-30  0.57
 2014-12-31  0.78
 2015-03-31  1.32
 2015-06-30  0.79
 2015-09-30  0.54
 2015-12-31  0.96
 2016-03-31  0.43
 2016-06-30  0.35
 2016-09-30  0.08
 2016-12-31  0.30
 2017-03-31  0.25
 2017-06-30 -0.23
 2017-09-30  0.16
 2017-12-31  0.44
 2018-03-31  0.09
 2018-06-30  1.26
 2018-09-30  0.48
 2018-12-31  0.15
 2019-03-31  0.75
 2019-06-30  0.01
 2019-09-30 -0.04
 2019-12-31  1.15
 2020-03-31  0.07
 2020-06-30  0.26
 2020-09-30  0.64
 2020-12-31  1.35
 2021-03-31  0.93
 2021-06-30  0.53
 2021-09-30  1.16
 2021-12-31  0.73
 2022-03-31  1.62
 2022-06-30  0.67
 2022-09-30 -0.29
 2022-12-31  0.62
 2023-03-31  0.71
 2023-06-30 -0.08
 2023-09-30  0.26
 2023-12-31  0.56
 2024-03-31  0.16,
 

In [42]:
# Salvando dados tratados
for key, df in resampled_indices.items():
    df.to_csv(f'clean_data\\{key}.csv', sep=';')
    print(f"Dados salvos {key} com sucesso!")

Dados salvos ipca com sucesso!
Dados salvos selic com sucesso!
Dados salvos igpm com sucesso!
Dados salvos vix com sucesso!
Dados salvos ibov com sucesso!
Dados salvos incc com sucesso!
Dados salvos iee com sucesso!
